# Fourth Coursework (Group X)
### Theoretical Foundations and Research Topics in Machine Learning

Follow the instructions in this notebook. Please remember to upload the filled in jupyter notebook as part of your final submission together with the PDF of the other tasks. It might be a good idea to also upload a PDF/HTML version of your jupyter notebook as this ensures that nothing gets lost during upload.

**IMPORTANT:** You are not allowed to use additional imports, i.e., you should implement all functionalities using NumPy only.

In [1]:
# Load packages
import random

import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import sklearn.datasets as dt

from sklearn.model_selection import train_test_split

# Display figure in the notebook
%matplotlib inline

#### Activation Functions & Loss (0.5 points)

In this section, you will have to implement three different activation functions (_Sigmoid_, _Tanh_, and _ReLU_). Please note that the method _forward()_ is the basic function, while the method _backward()_ should be used for the derivative of the function.

Additionally, you will have to implement the _Mean Squared Error_ loss function, which is defined as follows:

$$ MSE = \frac{1}{N} \sum^{N}_{i=1} (y_i - \hat{y_i})^2 $$.

You can use any functionality that is part of NumPy.

In [2]:
class Sigmoid():
    def forward(self, x):
        # Implement the sigmoid function
        ##### YOUR CODE HERE #####
        
        return 1 / (1 + np.exp(-x))
        

    def backward(self, x):
        # Implement the derivative of the sigmoid function
        ##### YOUR CODE HERE #####
        
        # derivative is df = f * (1 - f)
        return (1 / (1 + np.exp(-x))) * (1 - (1 / (1 + np.exp(-x))))

In [3]:
class TanH():
    def forward(self, x):
        # Implement the tanh function
        ##### YOUR CODE HERE #####
        
        return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))

    def backward(self, x):
        # Implement the derivative of the tanh function
        ##### YOUR CODE HERE #####
        
        tanh = (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
        return 1 - tanh**2

In [4]:
class ReLU():
    def forward(self, x):
        # Implement the relu function
        ##### YOUR CODE HERE #####
        
        return x * (x > 0)

    def backward(self, x):
        # Implement the derivative of the relu function
        ##### YOUR CODE HERE #####
    
        return 1. * (x > 0)

In [5]:
class MSE():
    def forward(self, y_pred, y_true):
        # Implement the mse function
        ##### YOUR CODE HERE #####
        
        return (np.square(y_pred - y_true)).mean()

    def backward(self, y_pred, y_true):
        # Implement the derivative of the mse function
        ##### YOUR CODE HERE #####
        
        return 2 * (y_pred - y_true).mean()

#### Multi-Layer Perceptron (2 points)

In this section, you will have to implement your very own _Multi-Layer Perceptron_. For the network architecture, we will consider only one hidden layer and no activation function for the output neuron (i.e. apply the activation function to the hidden layer but not to the output layer).

_Hint: You will need to compute the derivatives $\frac{\partial L}{\partial W_o}, \frac{\partial L}{\partial b_o}, \frac{\partial L}{\partial W_h}$ and $\frac{\partial L}{\partial b_h}$ (using the chain rule) as they are required for the backward pass._

In [322]:
class MLP():

    def __init__(self, input_size, hidden_size, output_size, activation_func = ReLU()):
        """
        Parameters
        ----------
        W_h: 
            weight matrix from input layer to hidden layer with size (input_size, hidden_size)
        b_h: 
            bias vector for the hidden layer with size (hidden_size)
        W_o: 
            weight matrix from hidden layer to output layer with size (hidden_size, output_size)
        b_o: 
            bias vector for the hidden layer with size (output_size)
        activation_func: 
            activation function of your choice
        """
        ##### YOUR CODE HERE #####
        
        self.W_h = np.random.random((input_size, hidden_size))
        self.b_h = np.random.random((1, hidden_size))
        
        self.W_o = np.random.random((hidden_size, output_size))
        self.b_o = np.random.random((1, output_size))
        
        self.activation_func = activation_func
        
    def forward(self, x):
        """
        forward pass of the MLP
        
        Parameters
        ----------
        x:
            input vector of size (input_size)
            
        Returns
        -------
        y:
            output vector of size (output_size)
        """
        ##### YOUR CODE HERE #####
        
        # first weights layer (hidden) (multiply + sum)
        f = np.dot(x, self.W_h) + self.b_h
        
        # activation function for the hidden layer
        f = self.activation_func.forward(f)
        
        # output weights layer (multiply + sum)
        y = np.dot(f, self.W_o) + self.b_o
        
        return y
        
    def forward_(self, x):
        """
        forward pass of the MLP with additional return values
        
        Paramaters
        ----------
        x:
            input vector of size (input_size)
            
        Returns
        -------
        y:
            output vector of size (output_size)
        h:
            activation of the hidden layer of size (hidden_size)
        z_h:
            pre-activation of the hidden layer of size (hidden_size)
            i.e., the input vector to the activation function
        """
        ##### YOUR CODE HERE #####
        
        # pre-activation of the hidden layer
        z_h = np.dot(x, self.W_h) + self.b_h    
        
        # activation of the hidden layer
        h = self.activation_func.forward(z_h)
                
        print(h)
        # output vector
        y = np.dot(h, self.W_o) + self.b_o
        
        return y, h, z_h
    
    def backward(self, x, h, z_h, dloss):
        """
        backward pass of the MLP
        
        Parameters
        ----------
        x:
            input vector of size (input_size)
        h:
            activation of the hidden layer of size (hidden_size)
        z_h:
            pre-activation of the hidden layer of size (hidden_size)
            i.e., the input vector to the activation function
        dloss:
            gradient of the loss function with respect to y_pred
            
        Returns
        -------
        grads:
            dictionary containing the elements
            - W_h: gradients for W_h
            - b_h: gradients for b_h
            - W_o: gradients for W_o
            - b_o: gradients for b_o
        """
        ##### YOUR CODE HERE #####
        
        n = x.shape[0]
        print(h.T.shape)
        print(len(dloss))

        dW_o = 1./n * np.dot(h.T, dloss)
        db_o = 1./n * np.sum(dloss, axis = 1, keepdims = True)
        dloss1 = np.dot(dloss, self.W_o.T) * (1 - np.power(h, 2))
        
        dW_h = 1./n * np.dot(x.T, dloss1)
        db_h = 1./n * np.sum(dloss1, axis = 1, keepdims = True)
        
        grads = {"W_h": dW_h,
                "b_h": db_h,
                "W_o": dW_o,
                "b_o": db_o}
        
        return grads

In [323]:
mlp = MLP(2, 3, 5, activation_func=ReLU())

In [324]:
x = np.random.random((2, 2))
x

array([[0.3765608 , 0.00212064],
       [0.54238187, 0.12584133]])

In [325]:
fwd = mlp.forward_(x)

[[1.13654587 0.91772233 0.82905093]
 [1.32447673 0.96372966 0.88182197]]


In [326]:
mlp.backward(x, fwd[1], fwd[2], [[0.5, 0.2, 0.5, 0.2, 0.4], [0.5, 0.2, 0.5, 0.2, 0.4]])

(3, 2)
2


{'W_h': array([[-0.2527643 ,  0.06230835,  0.12002154],
        [-0.04653189,  0.00590861,  0.01442553]]),
 'b_h': array([[ 0.11561423],
        [-0.21012907]]),
 'W_o': array([[0.61525565, 0.24610226, 0.61525565, 0.24610226, 0.49220452],
        [0.470363  , 0.1881452 , 0.470363  , 0.1881452 , 0.3762904 ],
        [0.42771823, 0.17108729, 0.42771823, 0.17108729, 0.34217458]]),
 'b_o': array([[0.9],
        [0.9]])}

#### Gradient Descent (2 points)

In this section, you will have to implement the training algorithm using _Gradient Descent_. 

While we provide you with the wrapper function, you need to implement the methods _evaluate()_ and _update()_, where the computation of the gradients and the weight update should be performed as part of the _update()_ method.

In [ ]:
def evaluate(data, model, loss_func):
    """
    function to evaluate the test data
    i.e., just forward pass and loss computation
    
    Parameters
    ----------
    data:
        input data containing X and y
    model:
        the initialized MLP model
    loss_func:
        loss function of your choice
    
    Returns
    -------
    losses:
        array containing all individual losses
        i.e., for each data sample
    """
    ##### YOUR CODE HERE #####
        
    
    
def update(data, model, loss_func, learning_rate):
    """
    function to calculate gradients and perform weight updates
    i.e., forward pass + loss computation + backward pass + weight update
    
    Parameters
    ----------
    data:
        input data containing X and y
    model:
        the initialized MLP model
    loss_func:
        loss function of your choice
    learning_rate:
        float value defining the learning rate
    
    Returns
    -------
    losses:
        array containing all individual losses
        i.e., for each data sample
    """
    ##### YOUR CODE HERE #####
        
    ##########################
    raise NotImplementedError

In [ ]:
# perform gradient descent, no ToDo for you
def gradient_descent(train_data, test_data, model, loss_func, epochs, learning_rate):
    valid_losses = evaluate(test_data, model, loss_func)
    print("Initial Validation: " + str(np.mean(valid_losses)))
    
    for epoch in range(epochs):
        train_losses = update(train_data, model, loss_func, learning_rate)
        valid_losses = evaluate(test_data, model, loss_func)
        
        print("Epoch " + str(epoch) + ": " + str(np.mean(train_losses)) + " Train Loss, " + str(np.mean(valid_losses)) + " Valid Loss")  

#### Train your model (0.5 points)

In this section, you will have to initialise your MLP (using defined hyperparameters) and train it on the provided data. For the training, you can, of course, simply use the _gradient$\_$descent()_ method. 

In [13]:
# Generating toy data, no ToDo for you
X, y = dt.make_regression(n_samples = 1000, n_features = 20)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

train_data = list(zip(X_train, y_train))
test_data = list(zip(X_test, y_test))

In [ ]:
# Initialize and train the model

##### YOUR CODE HERE #####

##########################

(1000, 20)